In [31]:
## load modules and functions

# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

# arguments
import sys
import argparse
import os

In [32]:
## parse arguments

parser = argparse.ArgumentParser()
parser.add_argument('-i', '--input_dir_path', type=str, help='Full path of the input directory')
parser.add_argument('-p', '--autoencoder_parameters', type=str, help='String containing specific parameters used in the autoencoder')
parser.add_argument('-f', '--feature', type=str, help='Which simulated feature==\'high\' genomic regions have been hypermutated (Phenotype==1), or not (Phenotype==0)')
parser.add_argument('-t', '--test_size', type=float, help='Fraction of the original data to be used as test (its complement will be the fraction used as training data)')
parser.add_argument('-s', '--seed', type=int, help='Seed for all functions that involve some randomness: train_test_split, RandomForestClassifier, RandomizedSearchCV')

# if interactive, pass some values manually
if 'ipykernel' in sys.modules:
    args = parser.parse_args(['-i',"/home/jovyan/franklin/malvarez/rf_inputs",
                              '-p',"8_1_1_100_200_relu_glorot.uniform_mean.squared.error_0.0001_Adam",
                              '-f','feature1', 
                              '-t','0.2', 
                              '-s','0'])
else:
    args = parser.parse_args()
    
input_dir_path,autoencoder_parameters,feature,test_size,seed = args.input_dir_path,args.autoencoder_parameters,args.feature,args.test_size,args.seed

In [33]:
## load and parse data

exposures = pd.read_csv(f'{input_dir_path}/{autoencoder_parameters}/{feature}_exposures.tsv', sep="\t")

# Training data is used to fit the model. The algorithm uses the training data to learn the relationship between the features and the target: USING TRAINING SAMPLES FROM AE
ae_training_samples = exposures[exposures['train_val'] == "training"].drop('train_val', axis=1)
# Test data is used to evaluate the performance of the model: USING VALIDATION SAMPLES FROM AE
ae_validation_samples = exposures[exposures['train_val'] == "validation"].drop('train_val', axis=1)

## Split both training and testing data into features (X, the exposures, the predictor) and target (y, the Phenotype column, that we want to predict)
X_train = ae_training_samples.drop('Phenotype', axis=1)
y_train = ae_training_samples['Phenotype']
X_test = ae_validation_samples.drop('Phenotype', axis=1)
y_test = ae_validation_samples['Phenotype']

Hyperparameter Tuning
---------------------

RandomizedSearchCV randomly search parameters within a range per hyperparameter. We define the hyperparameters to use and their ranges in the param_dist dictionary. In our case, we are using:

- n_estimators: the number of decision trees in the forest. Increasing this hyperparameter generally improves the performance of the model but also increases the computational cost of training and predicting.
- max_depth: the maximum depth of each decision tree in the forest. Setting a higher value for max_depth can lead to overfitting while setting it too low can lead to underfitting
- random_state is a seed for reproducing results

RandomizedSearchCV will train many models (defined by n_iter_ and save each one as variables, the code below creates a variable for the best model and prints the hyperparameters. In this case, we haven’t passed a scoring system to the function, so it defaults to accuracy. This function also uses cross validation, which means it splits the data into five equal-sized groups and uses 4 to train and 1 to test the result. It will loop through each group and give an accuracy score, which is averaged to find the best model.

In [34]:
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create an instance of the Random Forest model, with the default parameters
rf = RandomForestClassifier(random_state=seed,
                            oob_score=True)

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5,
                                 random_state=seed)

# Fit the random search object to our training data. We pass both the features and the target variable, so the model can learn.
rand_search.fit(X_train, y_train)

# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
#print('Best hyperparameters:', rand_search.best_params_)

In [46]:
## At this point, we have a trained Random Forest model, but we need to find out whether it is making accurate predictions
# Generate predictions with the best model
y_pred = best_rf.predict(X_test)

## Evaluate this model using accuracy, precision, and recall; we check the predictions against the actual values in the test set and count up how many the model got right
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
oob = 1 - best_rf.oob_score_
#print(f'Accuracy: {accuracy}')
#print(f'Precision: {precision}')
#print(f'Recall: {recall}')
#print(f'OOB error: {oob:.4f}')
       
## Get the importance of each feature, using the model’s internal score to find the best way to split the data within each decision tree
# Create a series containing feature importances from the model and feature names from the training data
feature_importances = pd.Series(best_rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
#feature_importances.plot.bar()

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
OOB error: 0.0000


In [68]:
## write output table
res = {'autoencoder_parameters': [autoencoder_parameters],
       'feature': [feature],
       'test_size': [test_size],
       'seed': [seed],
       'accuracy': [accuracy],
       'precision': [precision],
       'recall': [recall],
       'oob': [oob]}
# append AE components (the number of them varies between runs)
for i,feature_importance in enumerate(feature_importances.sort_index()):
    res[f'ae{i+1}'] = feature_importance

pd.DataFrame(res).to_csv(f'{autoencoder_parameters}_{feature}_{test_size}_{seed}_res.tsv', header=True, sep='\t', index=False)

Visualize the first 3 trees
---------------------------

Each tree image is limited to only showing the first few nodes. These trees can get very large and difficult to visualize. The colors represent the majority class of each node (boxes). The colors get darker the closer the node gets to being fully a category. Each node also contains the following information:

1- The variable name and value used for splitting

2- The % of total samples in each split

3- The % split between classes in each split

In [ ]:
'''
for i in range(3):
    tree = best_rf.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=X_train.columns,  
                               filled=True,  
                               max_depth=2, 
                               impurity=False, 
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)
'''

Confusion matrix
----------------

It plots what the model predicted against what the correct prediction was.
We can use this to understand the tradeoff between false positives (top right) and false negatives (bottom left)


In [ ]:
#ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test, y_pred), 
#                       display_labels=True).plot()